In [ ]:


import pandas as pd
from sqlalchemy import create_engine
import urllib
import os
from dotenv import load_dotenv

load_dotenv()

SERVER   = os.getenv("MSSQL_SERVER", "localhost,1433")
DB_NAME  = os.getenv("MSSQL_DB", "StudentDepressionDB")
USER     = os.getenv("MSSQL_UID", "SA")
PWD      = os.getenv("MSSQL_PWD")

CSV_PATH = "/Users/rifat-mac/Study/Project/Student Depression Data Analysis/Data/Raw/Depression+Student+Dataset.csv"
TABLE    = "student_data"
SCHEMA   = "dbo"

DRIVER_PATH = "/opt/homebrew/lib/libmsodbcsql.18.dylib"
if not os.path.exists(DRIVER_PATH):
    raise FileNotFoundError(" ODBC driver not found. Make sure msodbcsql18 is installed via Homebrew.")

os.environ["PATH"] += ":/opt/homebrew/bin:/opt/homebrew/lib"

conn_str = (
    f"DRIVER={DRIVER_PATH};"
    f"SERVER={SERVER};"
    f"DATABASE={DB_NAME};"
    f"UID={USER};PWD={PWD};"
    "TrustServerCertificate=yes;"
)

params = urllib.parse.quote_plus(conn_str)
engine = create_engine(
    f"mssql+pyodbc:///?odbc_connect={params}", 
    fast_executemany=True
)

df = pd.read_csv(CSV_PATH)
df.to_sql(TABLE, engine, schema=SCHEMA, if_exists="replace", index=False)
print(f"🚀 Uploaded to SQL: {SCHEMA}.{TABLE}  |  Rows: {len(df)}")

sample = pd.read_sql(f"SELECT TOP 5 * FROM {SCHEMA}.{TABLE};", engine)
display(sample)